In [2]:
import pandas as pd
from pathlib import Path

align_df = pd.read_csv(Path("librispeech/alignments/dev-clean/")/"alignments.csv")

In [ ]:
# filter out where 'text' == NAN
align_df = align_df[align_df["text"].notna()]
# filter out where 'text' == "<unk>""
align_df = align_df[align_df["text"] != "<unk>"]

# Group alignments by text 
grouped = align_df.groupby("text")

In [ ]:
for group in grouped:
    print(group[0])
    for i, file in enumerate(group[1].iterrows()):
        speaker = file[1]['filename'].split("/")[0]
        word_start = float(file[1]['word_start'])
        word_end = float(file[1]['word_end'])
        word = file[1]['text']
        print(f"{speaker} {word_start} {word_end} {word}")
    break



a
2412-153954-0007 5.35 5.43 a
2412-153954-0024 2.77 2.83 a
2412-153954-0000 3.51 3.57 a
2412-153954-0018 9.46 9.55 a
2412-153954-0013 0.93 0.99 a
2412-153954-0004 2.77 2.84 a
2412-153954-0009 5.45 5.53 a
2412-153954-0009 6.81 6.87 a
2412-153954-0003 5.11 5.18 a
2412-153954-0003 5.69 5.75 a
2412-153954-0020 1.18 1.25 a
2412-153954-0014 6.47 6.53 a
2412-153947-0016 3.59 3.66 a
2412-153947-0001 0.84 0.89 a
2412-153947-0001 1.62 1.67 a
2412-153947-0009 6.45 6.54 a
2412-153947-0009 7.42 7.49 a
2412-153947-0014 0.81 0.9 a
2412-153947-0015 6.39 6.45 a
2412-153948-0013 7.29 7.34 a
2412-153948-0004 10.46 10.53 a
2412-153948-0009 0.78 0.85 a
2412-153948-0006 4.02 4.1 a
2412-153948-0007 0.75 0.82 a
2412-153948-0007 4.1 4.15 a
2412-153948-0011 4.92 4.97 a
2078-142845-0012 2.86 2.93 a
2078-142845-0013 0.35 0.46 a
2078-142845-0050 2.88 2.94 a
2078-142845-0005 1.43 1.49 a
2078-142845-0005 11.11 11.19 a
2078-142845-0005 16.09 16.14 a
2078-142845-0005 27.92 27.98 a
2078-142845-0004 2.66 2.73 a
2078-14